# How to read that document

**You're intended to go through the low loss intermediate level notebook before completing that practical. Here we expect you to know how to perform data preprocessing and slicing.**

In this practical, you will learn how to use basic machine learning algorithm for the analysis of EELS data. If you are note familiar with Principal Component Analysis (PCA), we strongly suggest you to read the following. The next few questions will guide you through this practical : 

- How many components should you use to describe the data ? How many of them correspond to a physical signature ?
- How do the loadings obtained using PCA compare with the maps obtained by hand ?
- How do the loadings obtained using NMF compare with the maps obtained by hand ?

This notebook is largely inspired by the hyperspy documentation: http://hyperspy.org/hyperspy-doc/current/index.html

The data were kindly given by: Hugo Lourenço-Martins.

## Warning

**The methods presented here are very powerful but it may sometimes lead to wrong interpretations. If you plan to use them in your work we strongly recommend you to double check for the validity of the approach.** See for example: 

LICHTERT, Stijn et VERBEECK, Jo. Statistical consequences of applying a PCA noise filter on EELS spectrum images. Ultramicroscopy, 2013, vol. 125, p. 35-42.

# EELS statistics

Most of the tools in the hands of the analyst are based on statistical priors on the data. Thus, in this section we will describe the statistics of the EELS data.

The EELS processes are Poissonian in nature. Indeed, we observe a flow of electron over a given period of time and the inelastic scattering events are (in a good approximation) statistically independant from each other. It is often referred as shot noise.

Then the detection system can add up different kinds of noise such as beam jittering or dark current. We will detail here only the detector related noises.

## CCD detectors

The variance of the statistical noise of the CCD is given by the following formula : 

$$ Var(J(E)) = g + p J(E) $$

Where $J(E)$ is the detected electron current, following a Poisson statistics. $p$ is a conversion proportionality constant and $g$ is an additive gaussian noise originating from read-out noise and dark current. The variance is not constant over the channels of the detector.

## Direct detection

The direct electron detectors are mostly limited by shot noise, thus the noise variance becomes : 

$$ Var(J(E)) = p J(E) $$

The absence of $g$ drastically improve the performances of these detectors (especially at low doses). The variance is still not constant over the channels of the detectors.

In general, the data acquired using direct detection are more straightforward to analyze.

### References

EGERTON, Ray F. Electron energy-loss spectroscopy in the electron microscope. Springer Science & Business Media, 2011.

De la Peña Manchón, Francisco J. Advanced methods for Electron Energy Loss Spectroscopy core-loss analysis, PhD thesis, 2010

HART, James L., LANG, Andrew C., LEFF, Asher C., et al. Direct detection electron energy-loss spectroscopy: a method to push the limits of resolution and sensitivity. Scientific reports, 2017, vol. 7, no 1, p. 1-14

# PCA

Principal Component Analysis is a machine learning algorithm that can be used for data analysis or for denoising of multidimensional data. It is based on statistical principles.

## Principle

The data below are represented using two main axes, x and y. Each of those axes correspond to some variance of the data as it can be seen in the projection of the data on those axes (thick redlines). 

![image](images/random_data.png)

The PCA will reorganise the axes on which the data are represented. The axes are rotated so that they correspond to gradually decreasing variance of the data. It means that the first axis (arrow parallel the line) is represents the highest variance in the data (Thick blue line) and the second axis (arrow perpendicular to the line) represents lower variance (thick green line). 

![image](images/random_data_PCA.png)

For the data analysis, this reorganisation of the axes is useful. With the new representation we easily get that the main feature of the signal is the straight line and that the axis perpendicular to it represents mainly noise. **In general, after PCA you should determine which are the relevant axes (or components) describing your signal and discard the noisy ones.**

## The impact of noise statistics

The data below were generated with Poisson noise. The variance is not constant for all the data points. The reorganisation of the axes will not occur in the same way. Therefore, a correction of this effect is required. That is also why it is important to understand the statistics of the data you analyze.

![image](images/poisson_data.png)

⭐ When applying a least-square fit on data, the same principle applies. It is technically incorrect to fit data exhibiting Poisson statistics using a least-square method.

## Increasing the dimension of the data

The PCA principles apply even with data of higher dimension. For spectrum images, they can be represented as a collection of spectra (N pixels spectra). A point in 2D space corresponds to 2 coordinates (x,y). A point in 3D space corresponds to 3 coordinates (x,y,z). A spectrum can be seen as a point in E-dimensional space ($I_1, I_2, ..., I_E$).

![image](images/flat_spim.png)

PCA is going to decompose the data in two matrices. The first matricx called factors contains the vectors of the new representation, each column contains one spectrum-like axis. Each row of the second matrix (called loadings) correspond to the intensity of a given axis of the new representation. In the previous part with the line, the first line of the loadings will give out where each point is on the line. 

![image](images/decomposition.png)

For example, a spectral signature with high variance (such as the zero-loss peak) will have it's own axis in the new representation. Thus, there will be a corresponding loading describing the spatial evolution of the zero-loss peak intensity.



# Applying PCA

**We assume here that the data you load are aligned in energy and deconvoluted. Which is not the case in the example below.**

We use the `spim.decomposition` function to perform the PCA. Using the positional argument `spim.decomposition(True)`, the poissonian nature of the noise is taken into account (within some approximation). 

⭐ For fully taking into account the poisson statistics, you will need to use the maximum likelihood formulation of the algorihtm. This is more computationnally expensive though.

### References 

KEENAN, Michael R. et KOTULA, Paul G. Accounting for Poisson noise in the multivariate analysis of ToF‐SIMS spectrum images. Surface and Interface Analysis: An International Journal devoted to the development and application of techniques for the analysis of surfaces, interfaces and thin films, 2004, vol. 36, no 3, p. 203-212.

In [ ]:
%matplotlib qt
import hyperspy.api as hs

spim = hs.load('spim_low_loss.hspy')
spim.decomposition(True)

## Explained variance ratio

To help you determine the relevant components in your dataset, there is a tool in hyperspy to plot how much variance of the data correspond to each axis of the new representation. 

This plot organises the axes by decreasing variance. Often, this plot as an elbow shape. As a rule of thumb, the number of relevant components is approximately given by the position of the elbow.

In [ ]:
spim.plot_explained_variance_ratio()

## Plotting the results

You can plot the factors and loadings using `spim.plot_decomposition_factors` and `spim.plot_decomposition_loadings`. Here are a few useful options for display: 

- You can use an integer `spim.plot_decomposition_factors(3)`, which will display the 3 first factors. Or you can use a list `spim.plot_decomposition_factors([1,3,5])` which will display the factors 1, 3 and 5 only.

- You can plot everything in separated windows using the `same_window = False` keyword argument

In [ ]:
spim.plot_decomposition_factors(3,same_window = False)

In [ ]:
spim.plot_decomposition_loadings(3,same_window = False)

# NMF

## Principles

The Non-Negative Matrix Factorization is somewhat similar to PCA. It decomposes the data into factors and loadings, although they are not organized according to decreasing variance. The main difference is that the factors and loadings are constrained to positive values. 

We activate the NMF using the keyword argument `algorithm = 'NMF'`. The calculation is longer we limit beforehand the number of components using the keyword argument `output_dimension = n`, where `n` is the integer you choose. The algorithm might not converge with the default amount of iterations, you can use `max_iter=` to increase it.

**The PCA modifies the spim object. It is recommended to reload the data before performing NMF.**

⭐ Hyperspy has a flexible enough interface. You can use many different decomposition algorithms such as the ones of scikit-learn with more or less the same syntax. For example, the value of `algorithm=` can be any object that implements the `fit` and `transform` methods.

In [ ]:
%matplotlib qt
import hyperspy.api as hs

spim = hs.load('spim_low_loss.hspy')
spim.decomposition(True,algorithm = 'NMF',output_dimension = 3,max_iter = 2000)

## Plotting the results

In [ ]:
spim.plot_decomposition_factors(3)

In [ ]:
spim.plot_decomposition_loadings(3)